In [137]:
#1. Bibliotheken importieren
import mysql.connector as mysql
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly import tools
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
"""cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)"""

In [138]:
#Read all the credentials from external file
credentials = pd.read_csv('../credentials/credentials.csv')

database_adress = credentials['Host'].iloc[0]
database_user = credentials['User'].iloc[0]
database_password = credentials['Password'].iloc[0]
database_name = credentials['Database'].iloc[0]

print(database_user)

darth_vader


In [139]:
#Connect to Database
db = mysql.connect(host=database_adress, user=database_user, passwd=database_password, database=database_name)
cursor = db.cursor()

#Form and execute query
query = "SELECT * FROM movies;" ## Read from DB
cursor.execute(query)

#Load query in Dataframe and print it out
all_movies = pd.DataFrame(list(cursor), columns=['id', 'title', 'year'])

all_movies.iloc[0:10].style.hide_index()


/var/folders/4t/fcv79ymx7xd_k2n2z1njywxh0000gn/T/ipykernel_57606/1500676116.py:12: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`



id,title,year
1,Toy Story (1995),1995
2,Jumanji (1995),1995
3,Grumpier Old Men (1995),1995
4,Waiting to Exhale (1995),1995
5,Father of the Bride Part II (1995),1995
6,Heat (1995),1995
7,Sabrina (1995),1995
8,Tom and Huck (1995),1995
9,Sudden Death (1995),1995
10,GoldenEye (1995),1995


In [140]:
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [141]:
db_connection_str = f"mysql+pymysql://{database_user}:{database_password}@{database_adress}/{database_name}"
db_connection = create_engine(db_connection_str)

In [ ]:
all_movies = pd.read_sql('SELECT * FROM movies ORDER BY release_year', con=db_connection)

In [ ]:
all_movies.head(400)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

year = pd.DataFrame(all_movies['release_year'].drop_duplicates())
count = all_movies.groupby(['release_year'])['movieId'].count()

year_count = pd.merge(year, count, on=["release_year", "release_year"])
year_count.rename(columns = {'movieId':'count'}, inplace=True)

year_count.drop(0, inplace=True)
year_count.drop(year_count.index[len(year_count)-1], inplace=True)

ax.bar(year_count['release_year'], year_count['count'], color ='maroon',
      width = 0.4)

ax.set_title("Released movies over time")
#ax.set_xlim(1850,2018)

In [ ]:
year_count['release_year'].head()

In [ ]:
all_movies.groupby(['release_year']).head()#['movieId'].count()

In [ ]:
count = all_movies.groupby(['release_year'])['movieId'].count()

test_fig, test_ax = plt.subplots(figsize=(10,5))

count = count.drop([-1])
count = count.drop([2018])

test_ax.bar(count.index, count.values, color ='maroon')

test_ax.set_title("Released movies over time")

In [ ]:
all_movies.groupby(['release_year']).head()

In [ ]:
movies_with_genres = pd.read_sql("""SELECT m.movieId, m.title, g.genrename FROM movies as m 
INNER JOIN movies_genres as mg ON m.movieID = mg.movieId
INNER JOIN genres as g ON mg.genreId = g.genreId
ORDER BY m.movieId;""", con=db_connection)

In [ ]:
genres = pd.read_sql("""SELECT * FROM genres;""", con=db_connection)

In [ ]:
genres_count = movies_with_genres.groupby(['genrename'])['movieId'].count()

genres_count_fig, genres_count_ax = plt.subplots(figsize=(30,15))
#genres_count.head()
genres_count_ax.bar(genres_count.index, genres_count.values, color ='maroon')

In [ ]:
genres.count()

In [ ]:
genres_df = pd.read_sql('SELECT mg.movieId,mg.genreId,g.genrename FROM movies_genres AS mg INNER JOIN genres AS g ON mg.genreId=g.genreId;', con=db_connection)

In [ ]:
genres_df.groupby(['genrename'])['movieId'].count()

## (1) Released movies over time (in Plotly)

In [ ]:
all_movies = pd.read_sql('SELECT * FROM movies ORDER BY release_year', con=db_connection)

In [ ]:
count = all_movies.groupby(['release_year'])['movieId'].count()

noyear_count = count[-1]
print(f'Movies without year: {noyear_count} or {((noyear_count / count.values.sum())*100):.2f}%')

count = count.drop([-1])
count = count.drop([2018])

count_df = count.to_frame()

fig1 = px.bar(count_df, x=count_df.index, y='movieId', title='Released movies over time',
labels={
                     "release_year": "Year of release",
                     "movieId": "Amount of movies"
                 },
                )
            
fig1.update_layout(
    xaxis=dict(
        title='Year of release',
        titlefont_size=16
    ),
    yaxis=dict(
        title='Amount of movies',
        titlefont_size=16
    )
)
fig1.update_traces(marker_color='maroon')
fig1.show()

## (3) Genre share over time

Diagram Type: 100% Stacked Area Chart (https://codejock.com/products/chart/100-stacked-area-chart.asp)

**Challenges**
* [OK] Multiple genres
* [OK] Amount of genres to display

In [ ]:
movies_genres = pd.read_sql('SELECT m.movieId, m.title, m.release_year, g.genreId, g.genrename FROM movies as m inner join movies_genres as mg on m.movieId=mg.movieId inner join genres as g on mg.genreId=g.genreId order by m.movieId', con=db_connection)
movies_genres[:10]

In [ ]:
print(f'Average genres per film: {len(movies_genres) / len(movies_genres.movieId.unique()):.2f}')
print('\nCheck total distribution')
genres_total_dist = movies_genres.genrename.value_counts().to_frame()
genres_total_dist['share'] = genres_total_dist.apply(lambda x: (x / genres_total_dist.genrename.sum()*100))
print(genres_total_dist)

In [ ]:
#Calculating right share based on amount of genres
#For example: Jumanji (1995) has 3 genres assigned. Every genre should only count 1/3. 

#DISCLAIMER: can take up to 5min! not optimized yet!

genres_hist = pd.DataFrame(index=movies_genres.genrename.unique())

for year in range(1900,2018+1):
    q = movies_genres.query(f'release_year == {year}').copy()
    q['genre_vote'] = q['movieId'].apply(lambda x: (1/len(q.query(f'movieId == {x}'))))
    q = q.groupby(by='genrename').sum()
    q = q['genre_vote']
    q = q.round(decimals=2)
    genres_hist[year] = q


#old approach: absolute amount genre per year
"""
for year in range(1900,2018+1):
    genres_hist[year] = movies_genres.query(f'release_year == {year}').genrename.value_counts()
"""

In [ ]:
#Fill na with 0 and set type int
genres_hist.fillna(0,inplace=True)
genres_hist = genres_hist.astype(int)

#calculate relative share for every year
for year in range(1900,2018+1):
    sum = genres_hist[year].sum()
    genres_hist[year] = genres_hist[year].apply(lambda x: round((x / sum)*100,2))

#Transpose for easier data access
genres_hist = genres_hist.T
genres_hist

In [ ]:
pio.templates.default = "plotly_dark"

top10genres = genres_hist.loc[2018].sort_values(ascending=False).head(10).index.to_list()

color_seq = px.colors.qualitative.G10
color_seq_count = 0

fig3 = go.Figure()
for genre in top10genres:
    color = color_seq[color_seq_count]
    color_seq_count+=1
    fig3.add_trace(go.Scatter(x=genres_hist.index,
                         y=genres_hist[genre],
                         #fill='tozeroy', #fill down to xaxis
                         #fillcolor='orange',
                         mode='lines',
                         line={'dash': 'solid', 'color': color},
                         name=genre,
                         stackgroup='one',
                         ))
fig3.update_layout(
    xaxis=dict(
        title='Time in years',
        titlefont_size=16
    ),
    yaxis=dict(
        title='Share of genre [%]',
        titlefont_size=16
    ),
    yaxis_range=(0,100),
    xaxis_range=(1900,2018),
    title="Genre popularity over time"
)
fig3.add_vrect(x0=1900,x1=1920,line_width=0,fillcolor='black',opacity=0.4,annotation_text="not enough data",annotation_position="bottom left",
    annotation=dict(font_size=18,font_color='white')
)
fig3.show()

## (4) Find potentially suspect activity (bots)

In [19]:
#Only run to refresh local data copy

"""
users_with_one_rating = pd.read_sql('SELECT * FROM ratings as r WHERE (SELECT COUNT(*) FROM ratings as ra WHERE r.userId = ra.userId) = 1', con=db_connection)
"""

"\nusers_with_one_rating = pd.read_sql('SELECT * FROM ratings as r WHERE (SELECT COUNT(*) FROM ratings as ra WHERE r.userId = ra.userId) = 1', con=db_connection)\nall_ratings = pd.read_sql('SELECT * FROM ratings', con=db_connection)\n"

In [88]:
all_ratings = pd.read_sql('SELECT * FROM ratings', con=db_connection)

In [ ]:
users_with_one_rating.head()

In [178]:
all_ratings.head()

,rating_id,userId,movieId,rating,rating_date
0,1,1,307,3.5,2009-10-27 09:00:21
1,2,1,481,3.5,2009-10-27 09:04:16
2,3,1,1091,1.5,2009-10-27 09:04:31
3,4,1,1257,4.5,2009-10-27 09:04:20
4,5,1,1449,4.5,2009-10-27 09:01:04


In [179]:
len(all_ratings)

27753444

In [ ]:
len(users_with_one_rating)

In [ ]:
len(all_ratings)

In [ ]:
users_with_one_rating.sort_values(by=['movieId']).to_csv('users_with_only_one_rating.csv', index=False)

In [46]:
users_with_one_rating = pd.read_csv('users_with_only_one_rating.csv')

In [47]:
users_with_one_rating.head()

,rating_id,userId,movieId,rating,rating_date
0,19834829,202382,1,5.0,1997-05-20 06:50:22
1,25102792,256349,1,5.0,2017-08-24 03:49:46
2,15849334,161826,1,3.0,2016-12-17 15:13:43
3,21184239,216433,1,5.0,2015-11-19 04:07:21
4,4464337,45860,1,5.0,2001-11-24 12:06:59


In [7]:
len(users_with_one_rating)

5620

In [8]:
previous_movie = 0
movie_ratings = 0
movies_and_rating_amount = []

for index, row in users_with_one_rating.iterrows():
    if row['movieId'] == previous_movie:
        movie_ratings += 1
        if len(users_with_one_rating) - 1 == index:
            movies_and_rating_amount.append([int(previous_movie), int(movie_ratings)])
    else:
        movies_and_rating_amount.append([int(previous_movie), int(movie_ratings)])
        previous_movie = row['movieId']
        movie_ratings = 1

movies_ratings_from_user_with_only_one_rating = pd.DataFrame(movies_and_rating_amount, columns=['Movie Id', 'Amount'])

In [9]:
movies_ratings_from_user_with_only_one_rating.sort_values(by=['Amount'], ascending=False, inplace=True)
movies_ratings_from_user_with_only_one_rating.to_csv('movies_with_sus_ratings.csv', index=False)

In [15]:
#Append movie title to df

#Get relationship table between movieId and movie title
movies_ids_names = pd.read_sql('SELECT movieId, title FROM movies', con=db_connection)

#Drop last 0 entry to prevent error
movies_ratings_from_user_with_only_one_rating.drop(index=0, inplace=True, errors='ignore')

movies_ratings_from_user_with_only_one_rating['Movie Name'] = movies_ratings_from_user_with_only_one_rating['Movie Id'].apply(lambda x: movies_ids_names.query(f'movieId == {x}').title.values[0])

In [56]:
#pio.templates.default = "plotly_dark"

#iterate through movies with sus ratings to plot them (over time)
for i, row in movies_ratings_from_user_with_only_one_rating.head(20).iterrows():
    m_id = row['Movie Id']
    m_name = row['Movie Name']
    sus_rating = users_with_one_rating.query(f'movieId == {m_id}').copy()
    sus_rating.sort_values(by=['rating_date'], inplace=True)
    sus_rating_fig = px.scatter(sus_rating, x='rating_date', y='rating', title=f'Rating of movie <b>{m_name}</b> over time', range_x=['1998-01-01', '2019-01-01'], hover_data=['userId'])

    
    sus_rating_fig.show()


In [17]:
movies_ratings_from_user_with_only_one_rating.head()

,Movie Id,Amount,Movie Name
101,318,348,"Shawshank Redemption, The (1994)"
797,2858,113,American Beauty (1999)
718,2571,108,"Matrix, The (1999)"
116,356,71,Forrest Gump (1994)
494,1721,59,Titanic (1997)


In [19]:
import re

In [21]:
movies_ratings_from_user_with_only_one_rating['Movie Name'].iloc[0]

'Shawshank Redemption, The (1994)'

In [31]:
int(re.findall('\\d{4}(?!.*\\d)', movies_ratings_from_user_with_only_one_rating['Movie Name'].iloc[0])[0])

1994

In [41]:
movies_ratings_from_user_with_only_one_rating_new_releases = pd.DataFrame(columns=['Movie Id', 'Amount', 'Movie Name'])
i = 0

for index, movie in movies_ratings_from_user_with_only_one_rating.iterrows():
    list_year = re.findall('\\d{4}(?!.*\\d)', movie['Movie Name'])
    if len(list_year) == 0:
        continue
    if int(list_year[-1]) >= 2013:
        movies_ratings_from_user_with_only_one_rating_new_releases.loc[i] = movie
        i += 1

In [43]:
movies_ratings_from_user_with_only_one_rating_new_releases.head(10)

,Movie Id,Amount,Movie Name
0,109487,29,Interstellar (2014)
1,134130,23,The Martian (2015)
2,134853,17,Inside Out (2015)
3,164179,13,Arrival (2016)
4,176371,12,Blade Runner 2049 (2017)
5,152081,12,Zootopia (2016)
6,168252,11,Logan (2017)
7,177765,9,Coco (2017)
8,116797,7,The Imitation Game (2014)
9,122904,6,Deadpool (2016)


In [44]:
len(movies_ratings_from_user_with_only_one_rating_new_releases)

271

In [54]:
sus_rating = users_with_one_rating.query(f'movieId == 109487')

29

In [153]:
def return_list_of_binned_ratings(list_of_ratings, ratings):
    ratings_for_movie_binned_lst = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #ratings_idex_dict = {ratings[0] : 0, ratings[1] : 1, 4.0 : 2, 3.5 : 3, 3.0 : 4, 2.5 : 5, 2.0 : 6, 1.5: 7, 1.0 : 8, 0.5 : 9}

    for rating in ratings:
        index = list_of_ratings.index(rating)
        ratings_for_movie_binned_lst[index] += 1

    return ratings_for_movie_binned_lst


In [154]:

list_of_ratings = [5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5]  
    
return_list_of_binned_ratings(list_of_ratings, list(users_with_one_rating.query(f'movieId == 109487').copy()['rating']))

[21, 3, 3, 1, 0, 1, 0, 0, 0, 0]

In [155]:
def remove_sus_ratings(all_ratings_of_movie, users_with_one_rating_movie):
    #all_ratings_of_movie = all_ratings.query(f'movieId == 109487').reset_index(drop=True).copy()

    #users_with_one_rating_movie = users_with_one_rating.query(f'movieId == 109487')

    for index_all_ratings, rating in all_ratings_of_movie.iterrows():
        for index_one_rating, user_with_one_rating in users_with_one_rating_movie.iterrows():
            if rating['rating_id'] == user_with_one_rating['rating_id']:
                all_ratings_of_movie.drop(index_all_ratings, inplace=True)
    
    return all_ratings_of_movie




In [146]:
all_ratings_of_movie = all_ratings.query(f'movieId == 109487').reset_index(drop=True).copy()
print(all_ratings_of_movie.head())

   rating_id  userId  movieId  rating         rating_date
0       1273      14   109487     5.0 2015-09-13 06:38:38
1       2079      28   109487     5.0 2017-01-07 08:46:07
2       2261      34   109487     5.0 2016-09-03 11:46:48
3       3830      48   109487     5.0 2018-05-28 03:21:56
4       5820      65   109487     5.0 2015-02-02 02:50:05


In [136]:
#users_with_one_rating_movie = users_with_one_rating.query(f'movieId == 109487').copy()
#all_ratings_of_movie = all_ratings.query(f'movieId == 109487').copy()

#print(return_list_of_binned_ratings_without_sus(all_ratings.query(f'movieId == 109487').copy(), users_with_one_rating.query(f'movieId == 109487').copy()))

None


In [80]:
for i, row in movies_ratings_from_user_with_only_one_rating_new_releases.head(20).iterrows():
    m_id = row['Movie Id']
    m_name = row['Movie Name']
    list_of_ratings = [5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5]
    amount_of_ratings = return_list_of_binned_ratings(list_of_ratings, list(users_with_one_rating.query(f'movieId == {m_id}').copy()['rating']))
    d = {'rating':list_of_ratings,'amount of ratigns': amount_of_ratings}
    sus_rating_amount = pd.DataFrame(d)
    sus_rating_fig = px.bar(sus_rating_amount, x='rating', y='amount of ratigns', title=f'Rating Score over Percentage of movie <b>{m_name}</b>') #, range_x=['1998-01-01', '2019-01-01'], hover_data=['userId']
    sus_rating_fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0.5,
        dtick = 0.5
        )
    )

    
    sus_rating_fig.show()

In [ ]:
import plotly.graph_objects as go

In [176]:
list_of_rating_scores = [5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5]

for i, row in movies_ratings_from_user_with_only_one_rating_new_releases.head(20).iterrows():
    m_id = row['Movie Id']
    m_name = row['Movie Name']

    users_with_one_rating_movie = users_with_one_rating.query(f'movieId == {m_id}').copy()
    length_users_with_one_rating_movie = len(users_with_one_rating_movie)
    all_ratings_of_movie = all_ratings.query(f'movieId == {m_id}').copy()

    binned_ratings_sus = return_list_of_binned_ratings(list_of_rating_scores, list(users_with_one_rating_movie['rating']))

    ratings_not_sus = remove_sus_ratings(all_ratings_of_movie, users_with_one_rating_movie)
    length_ratings_not_sus = len(ratings_not_sus)
    binned_ratings_not_sus = return_list_of_binned_ratings(list_of_rating_scores, list(ratings_not_sus['rating']))

    binned_ratings_sus_pecentage = [binned_rating_sus / length_users_with_one_rating_movie for binned_rating_sus in binned_ratings_sus]
    binned_ratings_not_sus_pecentage = [binned_rating_not_sus / length_ratings_not_sus for binned_rating_not_sus in binned_ratings_not_sus]


    fig = go.Figure(data=[
        go.Bar(name='Ratings from users who rated only one movie', x=list_of_rating_scores, y=binned_ratings_sus_pecentage),
        go.Bar(name='Ratings from users who rated multiple movies', x=list_of_rating_scores, y=binned_ratings_not_sus_pecentage)
    ])
    fig.update_xaxes(tickvals=list_of_rating_scores)
    
    fig.update_layout(title_text=f'Comparison of ratings from user with only one rating compare to all other users in percentage for the movie <b>{m_name}</b>')
    fig.update_layout(barmode='group')  
    fig.show()

In [199]:
users_with_one_rating.head()

,rating_id,userId,movieId,rating,rating_date
0,19834829,202382,1,5.0,1997-05-20 06:50:22
1,25102792,256349,1,5.0,2017-08-24 03:49:46
2,15849334,161826,1,3.0,2016-12-17 15:13:43
3,21184239,216433,1,5.0,2015-11-19 04:07:21
4,4464337,45860,1,5.0,2001-11-24 12:06:59


In [185]:
users_with_one_rating['rating_date'].iloc[0][0:4]

'1997'

In [259]:
def calcluate_rating_amount_grouped_by_year(ratings_df):
    year_ratings_dict = {}

    for index, rating in ratings_df.iterrows():
        if int(str(rating['rating_date'])[0:4]) in year_ratings_dict:
            year_ratings_dict[int(str(rating['rating_date'])[0:4])] += 1
        else:
            year_ratings_dict[int(str(rating['rating_date'])[0:4])] = 1
    
    year_ratings_dict_sorted = {}

    for key in sorted(year_ratings_dict):
        year_ratings_dict_sorted[key] = year_ratings_dict[key]

    year_rating_dict = year_ratings_dict_sorted

    year_ratings = pd.DataFrame.from_dict(year_rating_dict, orient='index', columns=['Amount of ratings'])
    year_ratings.index.name = 'Year'
    year_ratings = year_ratings.reset_index()
    
    return year_ratings

In [267]:
years_users_with_one_rating = calcluate_rating_amount_grouped_by_year(users_with_one_rating)
sus_rating_grouped_by_year = px.bar(years_users_with_one_rating, x='Year', y='Amount of ratings', title=f'Rating amount of users with only one rating')
sus_rating_grouped_by_year.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1997,
        dtick = 1
        ))
sus_rating_grouped_by_year.show()

In [ ]:
years_all_ratings = calcluate_rating_amount_grouped_by_year(all_ratings)

In [265]:
ratings_grouped_by_year = px.bar(years_all_ratings, x='Year', y='Amount of ratings', title=f'All ratings for each year')
ratings_grouped_by_year.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1997,
        dtick = 1
        ))
sus_rating_grouped_by_year.show()

In [253]:
str(all_ratings['rating_date'].iloc[0])

'2009-10-27 09:00:21'

In [275]:
fig = go.Figure(data=[
        go.Bar(name='Ratings from users who rated only one movie', x=years_users_with_one_rating['Year'], y=years_users_with_one_rating['Amount of ratings'] / len(users_with_one_rating)),
        go.Bar(name='All ratings', x=years_all_ratings['Year'], y=years_all_ratings['Amount of ratings'] / len(all_ratings))
    ])
fig.update_xaxes(tickvals=list_of_rating_scores)
    
fig.update_layout(title_text=f'Comparison of all ratings and ratings made from accounts with only one rating in percentage')
fig.update_layout(barmode='group')  
fig.show()

In [271]:
years_users_with_one_rating['Amount of ratings']

0     176
1     361
2     577
3     144
4      96
5      63
6      44
7      43
8      51
9      75
10    136
11    236
12    207
13    278
14    305
15    272
16    299
17    305
18    499
19    514
20    578
21    361
Name: Amount of ratings, dtype: int64

22